In [1]:
%load_ext nb_black
%matplotlib inline

<IPython.core.display.Javascript object>

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import os
import matplotlib.pyplot as plt
import pmdarima as pm

warnings.filterwarnings("ignore")  # Ignoring warnings to reduce text spam.

sns.set(rc={"figure.figsize": (15, 8.27)})
pd.set_option("display.float_format", "{:,}".format)
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_rows", None)

<IPython.core.display.Javascript object>

In [3]:
#df['quantity'].str.replace(',','.').astype(float)

<IPython.core.display.Javascript object>

In [4]:
def retrieve_data():
    os.chdir('..')
    cwd = os.getcwd()
    if 'data' in os.listdir():
        os.chdir(cwd + '\\data')
        df = pd.read_csv('data.csv', sep=';', decimal=',')
        df['date'] = pd.to_datetime(df['date'])
        df['quantity'] = df['quantity'].round().astype('int64')
        #df.set_index('date', inplace=True)
        return df

<IPython.core.display.Javascript object>

In [5]:
df = retrieve_data()

<IPython.core.display.Javascript object>

In [6]:
def get_date_vals(frame):
    frame['year'] = frame['date'].dt.year
    frame['month'] = frame['date'].dt.month
    frame['week'] = frame['date'].dt.week
    return frame

<IPython.core.display.Javascript object>

In [7]:
tdf = get_date_vals(df)

<IPython.core.display.Javascript object>

In [8]:
week = tdf.groupby(['year','month','week'], as_index=False).sum()[['year','month','week','quantity']]

<IPython.core.display.Javascript object>

In [19]:
df.head()

,date,quantity,year,month,week
0,2014-01-03,339507,2014,1,1
1,2014-01-10,775969,2014,1,2
2,2014-01-17,764408,2014,1,3
3,2014-01-24,731060,2014,1,4
4,2014-01-31,835573,2014,1,5


<IPython.core.display.Javascript object>

In [14]:
x = mth['quantity']

<IPython.core.display.Javascript object>

In [16]:
adf = pm.arima.ADFTest(alpha=0.05)
adf.should_diff(x)

(0.01, False)

<IPython.core.display.Javascript object>